In [1]:
import time

start = time.process_time()

from pyspark.sql import SparkSession 
spark = SparkSession.builder.getOrCreate() 
sc = spark.sparkContext

from pyspark.sql import Row
from pyspark.sql import HiveContext
sqlContext = HiveContext(sc)

import sqlite3
import pandas as pd
from pprint import pprint

conn = None
try:
    conn = sqlite3.connect('database.sqlite')
except Error as e:
    print(e)


In [2]:
# pdTemp = pd.read_sql_query("SELECT * From Player;",conn)
# print(pdTemp.head(10))
Player = spark.createDataFrame(pd.read_sql_query("SELECT * From Player;",conn))
Player.persist()
Player.createOrReplaceTempView("Player")
Player_Attributes = spark.createDataFrame(pd.read_sql_query("SELECT * From Player_Attributes;",conn))
Player_Attributes.createOrReplaceTempView("Player_Attributes")
Match = spark.createDataFrame(pd.read_sql_query("SELECT * From Match;",conn))
Match.persist()
Match.createOrReplaceTempView("Match")
League = spark.createDataFrame(pd.read_sql_query("SELECT * From League;",conn))
League.createOrReplaceTempView("League")
Country = spark.createDataFrame(pd.read_sql_query("SELECT * From Country;",conn))
League.persist()
Country.createOrReplaceTempView("Country")
Team = spark.createDataFrame(pd.read_sql_query("SELECT * From Team;",conn))
Team.createOrReplaceTempView("Team")
Team_Attributes = spark.createDataFrame(pd.read_sql_query("SELECT * From Team_Attributes;",conn))
Team_Attributes.createOrReplaceTempView("Team_Attributes")
Player.show()

+---+-------------+--------------------+------------------+-------------------+------+------+
| id|player_api_id|         player_name|player_fifa_api_id|           birthday|height|weight|
+---+-------------+--------------------+------------------+-------------------+------+------+
|  1|       505942|  Aaron Appindangoye|            218353|1992-02-29 00:00:00|182.88|   187|
|  2|       155782|     Aaron Cresswell|            189615|1989-12-15 00:00:00|170.18|   146|
|  3|       162549|         Aaron Doran|            186170|1991-05-13 00:00:00|170.18|   163|
|  4|        30572|       Aaron Galindo|            140161|1982-05-08 00:00:00|182.88|   198|
|  5|        23780|        Aaron Hughes|             17725|1979-11-08 00:00:00|182.88|   154|
|  6|        27316|          Aaron Hunt|            158138|1986-09-04 00:00:00|182.88|   161|
|  7|       564793|          Aaron Kuhl|            221280|1996-01-30 00:00:00|172.72|   146|
|  8|        30895|        Aaron Lennon|            152747|1

In [3]:
Team_Attributes.printSchema()

root
 |-- id: long (nullable = true)
 |-- team_fifa_api_id: long (nullable = true)
 |-- team_api_id: long (nullable = true)
 |-- date: string (nullable = true)
 |-- buildUpPlaySpeed: long (nullable = true)
 |-- buildUpPlaySpeedClass: string (nullable = true)
 |-- buildUpPlayDribbling: double (nullable = true)
 |-- buildUpPlayDribblingClass: string (nullable = true)
 |-- buildUpPlayPassing: long (nullable = true)
 |-- buildUpPlayPassingClass: string (nullable = true)
 |-- buildUpPlayPositioningClass: string (nullable = true)
 |-- chanceCreationPassing: long (nullable = true)
 |-- chanceCreationPassingClass: string (nullable = true)
 |-- chanceCreationCrossing: long (nullable = true)
 |-- chanceCreationCrossingClass: string (nullable = true)
 |-- chanceCreationShooting: long (nullable = true)
 |-- chanceCreationShootingClass: string (nullable = true)
 |-- chanceCreationPositioningClass: string (nullable = true)
 |-- defencePressure: long (nullable = true)
 |-- defencePressureClass: strin

In [4]:
Player.printSchema()

root
 |-- id: long (nullable = true)
 |-- player_api_id: long (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_fifa_api_id: long (nullable = true)
 |-- birthday: string (nullable = true)
 |-- height: double (nullable = true)
 |-- weight: long (nullable = true)



In [7]:
Player.createOrReplaceTempView("Player")
results = sqlContext.sql("""SELECT player_name AS `Player Name`, birthday from Player \
            WHERE SUBSTRING(birthday,0,4) IN ('1987','1988','1989','1990') ORDER BY birthday""")
print(results.count())
results.show()

2943
+--------------------+-------------------+
|         Player Name|           birthday|
+--------------------+-------------------+
|       Abdullah Omar|1987-01-01 00:00:00|
|         Ben Alnwick|1987-01-01 00:00:00|
|           Loic Remy|1987-01-02 00:00:00|
|       Robert Milsom|1987-01-02 00:00:00|
|           Celestino|1987-01-02 00:00:00|
|           Luis Dias|1987-01-03 00:00:00|
|        Chris Turner|1987-01-03 00:00:00|
|        Dani Estrada|1987-01-03 00:00:00|
|    Przemyslaw Tyton|1987-01-04 00:00:00|
|           Kay Voser|1987-01-04 00:00:00|
|       Hans Martinez|1987-01-04 00:00:00|
|       Danny Simpson|1987-01-04 00:00:00|
|        Migjen Basha|1987-01-05 00:00:00|
|   Esteban Casagolda|1987-01-05 00:00:00|
|Claudio Lustenberger|1987-01-06 00:00:00|
|       Davide Astori|1987-01-07 00:00:00|
|  Michael McGlinchey|1987-01-07 00:00:00|
|      Stefan Babovic|1987-01-07 00:00:00|
|         Lucas Leiva|1987-01-09 00:00:00|
|     Michele Rinaldi|1987-01-09 00:00:00|
+-----

In [ ]:
results = spark.sql("""SELECT first(Country.name) AS `Country`,first(League.name) AS `League Name`, \
                SUM(home_team_goal+away_team_goal) AS `Total Goals Scored` FROM Match, Country, League \
                WHERE Country.id=Match.country_id AND League.id=Match.league_id \
                GROUP BY Match.country_id,Match.league_id""")
results.count()